# Fine-Tuning QA model

https://www.youtube.com/watch?v=3XiJrn_8F9Q&t=1197s <br>
https://www.youtube.com/watch?v=6ppxd9lp-X0


In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

#Reference: https://huggingface.co/deepset/roberta-base-squad2

model_name = "deepset/roberta-base-squad2"

""" # a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input) """

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
#looking at RoBerta Question Answering
model

How to fine-tune a QA model
- we need GPU
- building a training script


In [ ]:
import requests
import json

url = "https://raw.githubusercontent.com/AlinZohari/InformationExtraction/main/data/QA_model/train.json"
response = requests.get(url)
train = response.json()

In [ ]:
#looking at the train dataset
train

In [ ]:
import requests
import json

url = "https://raw.githubusercontent.com/AlinZohari/InformationExtraction/main/data/QA_model/validation.json"
response = requests.get(url)
validation = response.json()

In [ ]:
#looking atthe validation dataset
validation

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

## Training

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [ ]:
# Let's start training!
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=0.01,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=11,
    num_train_epochs=1,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["validations"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()